In [55]:
from operator import add
from pyspark.sql import SparkSession
import sys
import pyspark.sql.functions
import pandas as pd
from pyspark.sql.types import *
from pyspark.mllib.classification import SVMModel
from pyspark.mllib.classification import SVMWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.tree import RandomForest 

# spark = SparkSession.builder.appName("repurchase").getOrCreate()

In [95]:
import numpy 

In [57]:
df_train = pd.read_csv("train_format1.csv")
df_test = pd.read_csv("test_format1.csv")
user_info = pd.read_csv("user_info_format1.csv")
user_log = pd.read_csv('user_log_format1.csv')



# user_log = spark.read.options(header='True', inferSchema='False', delimiter=',')\
# .csv("user_log_format1.csv").distinct()

In [58]:
# df2 = spark.read.options(header='True', inferSchema='False', delimiter=',')\
# .csv("train_format1.csv").distinct()

print(df_test.shape,df_train.shape)
print(user_info.shape,user_log.shape)

(261477, 3) (260864, 3)
(424170, 3) (54925330, 7)


In [59]:
#特征工程
df_train[df_train['label'] == 1]

,user_id,merchant_id,label
2,34176,4356,1
7,231552,3828,1
53,306816,1489,1
57,176256,3323,1
59,307584,1340,1
...,...,...,...
260747,208511,2592,1
260793,87935,1964,1
260794,87935,3734,1
260799,350591,4394,1


In [60]:
#age_range gender
df_train = pd.merge(df_train,user_info,on="user_id",how="left")
df_train.head()

,user_id,merchant_id,label,age_range,gender
0,34176,3906,0,6.0,0.0
1,34176,121,0,6.0,0.0
2,34176,4356,1,6.0,0.0
3,34176,2217,0,6.0,0.0
4,230784,4818,0,0.0,0.0


In [61]:
#category
categories_temp = user_log.groupby([user_log["user_id"],user_log["seller_id"],user_log["cat_id"]]).count().reset_index()[["user_id","seller_id","cat_id"]]

In [62]:
categories_temp.head(20)

,user_id,seller_id,cat_id
0,1,471,389
1,1,739,1252
2,1,925,1023
3,1,1019,992
4,1,1156,1256
5,1,2245,276
6,1,4026,1252
7,1,4177,1252
8,1,4335,389
9,2,420,602


In [63]:
categories_temp1 = categories_temp.groupby([categories_temp["user_id"],categories_temp["seller_id"]]).count().reset_index()

In [64]:
categories_temp1.rename(columns={"seller_id":"merchant_id","cat_id":"categories"},inplace=True)

In [65]:
df_train = pd.merge(df_train,categories_temp1,on=["user_id","merchant_id"],how="left")

In [66]:
df_train.head(10)

,user_id,merchant_id,label,age_range,gender,categories
0,34176,3906,0,6.0,0.0,6
1,34176,121,0,6.0,0.0,1
2,34176,4356,1,6.0,0.0,1
3,34176,2217,0,6.0,0.0,1
4,230784,4818,0,0.0,0.0,1
5,362112,2618,0,4.0,1.0,1
6,34944,2051,0,5.0,0.0,1
7,231552,3828,1,5.0,0.0,15
8,231552,2124,0,5.0,0.0,1
9,232320,1168,0,4.0,1.0,1


In [67]:
#one_clicks、shopping_carts、purchase_times、favourite_times

one_clicks_temp = user_log.groupby([user_log["user_id"],user_log["seller_id"],user_log["action_type"]]).count().reset_index()[["user_id","seller_id","action_type","item_id"]]
one_clicks_temp.head(10)

,user_id,seller_id,action_type,item_id
0,1,471,0,1
1,1,739,0,1
2,1,925,0,3
3,1,925,2,1
4,1,1019,0,10
5,1,1019,2,4
6,1,1156,0,1
7,1,2245,0,5
8,1,4026,0,4
9,1,4026,2,1


In [68]:
one_clicks_temp.rename(columns={"seller_id":"merchant_id","item_id":"times"},inplace=True)

In [69]:
one_clicks_temp["one_clicks"] = one_clicks_temp["action_type"] == 0

In [70]:
one_clicks_temp["one_clicks"] = one_clicks_temp["one_clicks"] * one_clicks_temp["times"]
one_clicks_temp.head(10)

,user_id,merchant_id,action_type,times,one_clicks
0,1,471,0,1,1
1,1,739,0,1,1
2,1,925,0,3,3
3,1,925,2,1,0
4,1,1019,0,10,10
5,1,1019,2,4,0
6,1,1156,0,1,1
7,1,2245,0,5,5
8,1,4026,0,4,4
9,1,4026,2,1,0


In [71]:
one_clicks_temp["shopping_carts"] = one_clicks_temp["action_type"] == 1

In [72]:
one_clicks_temp["shopping_carts"] = one_clicks_temp["shopping_carts"] * one_clicks_temp["times"]
one_clicks_temp.head(10)

,user_id,merchant_id,action_type,times,one_clicks,shopping_carts
0,1,471,0,1,1,0
1,1,739,0,1,1,0
2,1,925,0,3,3,0
3,1,925,2,1,0,0
4,1,1019,0,10,10,0
5,1,1019,2,4,0,0
6,1,1156,0,1,1,0
7,1,2245,0,5,5,0
8,1,4026,0,4,4,0
9,1,4026,2,1,0,0


In [73]:
one_clicks_temp["purchase_times"] = one_clicks_temp["action_type"] == 2

In [74]:
one_clicks_temp["purchase_times"] = one_clicks_temp["purchase_times"] * one_clicks_temp["times"]
one_clicks_temp.head(10)

,user_id,merchant_id,action_type,times,one_clicks,shopping_carts,purchase_times
0,1,471,0,1,1,0,0
1,1,739,0,1,1,0,0
2,1,925,0,3,3,0,0
3,1,925,2,1,0,0,1
4,1,1019,0,10,10,0,0
5,1,1019,2,4,0,0,4
6,1,1156,0,1,1,0,0
7,1,2245,0,5,5,0,0
8,1,4026,0,4,4,0,0
9,1,4026,2,1,0,0,1


In [75]:
one_clicks_temp["favourite_times"] = one_clicks_temp["action_type"] == 3

In [76]:
one_clicks_temp["favourite_times"] = one_clicks_temp["favourite_times"] * one_clicks_temp["times"]
one_clicks_temp.head(10)


,user_id,merchant_id,action_type,times,one_clicks,shopping_carts,purchase_times,favourite_times
0,1,471,0,1,1,0,0,0
1,1,739,0,1,1,0,0,0
2,1,925,0,3,3,0,0,0
3,1,925,2,1,0,0,1,0
4,1,1019,0,10,10,0,0,0
5,1,1019,2,4,0,0,4,0
6,1,1156,0,1,1,0,0,0
7,1,2245,0,5,5,0,0,0
8,1,4026,0,4,4,0,0,0
9,1,4026,2,1,0,0,1,0


In [78]:
four_features = one_clicks_temp.groupby([one_clicks_temp["user_id"],one_clicks_temp["merchant_id"]]).sum().reset_index()
four_features.head(10)

,user_id,merchant_id,action_type,times,one_clicks,shopping_carts,purchase_times,favourite_times
0,1,471,0,1,1,0,0,0
1,1,739,0,1,1,0,0,0
2,1,925,2,4,3,0,1,0
3,1,1019,2,14,10,0,4,0
4,1,1156,0,1,1,0,0,0
5,1,2245,0,5,5,0,0,0
6,1,4026,2,5,4,0,1,0
7,1,4177,0,1,1,0,0,0
8,1,4335,0,1,1,0,0,0
9,2,420,2,26,23,0,3,0


In [79]:
four_features = four_features.drop(["action_type","times"], axis=1)

In [80]:
df_train = pd.merge(df_train,four_features,on=["user_id","merchant_id"],how="left")
df_train.head(10)

,user_id,merchant_id,label,age_range,gender,categories,one_clicks,shopping_carts,purchase_times,favourite_times
0,34176,3906,0,6.0,0.0,6,36,0,1,2
1,34176,121,0,6.0,0.0,1,13,0,1,0
2,34176,4356,1,6.0,0.0,1,12,0,6,0
3,34176,2217,0,6.0,0.0,1,1,0,1,0
4,230784,4818,0,0.0,0.0,1,7,0,1,0
5,362112,2618,0,4.0,1.0,1,0,0,1,0
6,34944,2051,0,5.0,0.0,1,2,0,1,0
7,231552,3828,1,5.0,0.0,15,78,0,5,0
8,231552,2124,0,5.0,0.0,1,6,0,1,0
9,232320,1168,0,4.0,1.0,1,2,0,1,1


In [81]:
df_train = df_train.fillna(method='ffill')
# 缺失值向前填充

In [ ]:
#resAge.write.csv("PycharmProjects/FBD-Final/output/test.csv" ) 似乎不行，每一行都是一个文件

In [ ]:
# resAge.toPandas().to_csv('PycharmProjects/FBD-Final/output/ageRatioSQL.csv')

In [82]:
df_train.to_csv("特征构建1.csv")

In [ ]:
spark = SparkSession.builder.appName("repurchase").getOrCreate()

In [169]:
data = spark.read.options(header='True', inferSchema='False', delimiter=',')\
.csv("特征构建1.csv").rdd

In [171]:
headerTrain = data.first()

Row(_c0='0', user_id='34176', merchant_id='3906', label='0', age_range='6.0', gender='0.0', categories='6', one_clicks='36', shopping_carts='0', purchase_times='1', favourite_times='2')

In [172]:
data1 = data.filter(lambda row:row != headerTrain)

In [173]:
data.top(10)

[Row(_c0='99999', user_id='82342', merchant_id='4382', label='0', age_range='4.0', gender='0.0', categories='1', one_clicks='9', shopping_carts='0', purchase_times='1', favourite_times='1'),
 Row(_c0='99998', user_id='344230', merchant_id='114', label='0', age_range='0.0', gender='0.0', categories='2', one_clicks='7', shopping_carts='0', purchase_times='2', favourite_times='0'),
 Row(_c0='99997', user_id='344230', merchant_id='4394', label='0', age_range='0.0', gender='0.0', categories='2', one_clicks='7', shopping_carts='0', purchase_times='1', favourite_times='0'),
 Row(_c0='99996', user_id='147622', merchant_id='10', label='0', age_range='6.0', gender='0.0', categories='2', one_clicks='16', shopping_carts='0', purchase_times='1', favourite_times='0'),
 Row(_c0='99995', user_id='409510', merchant_id='4601', label='0', age_range='2.0', gender='0.0', categories='4', one_clicks='23', shopping_carts='1', purchase_times='1', favourite_times='10'),
 Row(_c0='99994', user_id='409510', merch

In [98]:
data2 = data1.collect()

# trainData = data1.map(lambda x:LabeledPoint(label=x[3],features=x[4:]))


In [99]:
trainData = sc.parallelize(data2).map(lambda x:LabeledPoint(label=x[3],features=x[4:]))

In [100]:

print("训练集数量:{}".format(trainData.count()))

训练集数量:260863


In [101]:

svm = SVMWithSGD.train(sc.parallelize(trainData.collect()), iterations=10)

In [131]:
prediction = svm.predict(trainData.first().features)
print("真实值:{},预测值{}".format(prediction,trainData.first().label))


真实值:0,预测值0.0


In [103]:
#构造测试数据集
#构建特征
df_test = pd.merge(df_test,user_info,on="user_id",how="left")
df_test = pd.merge(df_test,categories_temp1,on=["user_id","merchant_id"],how="left")
df_test = pd.merge(df_test,four_features,on=["user_id","merchant_id"],how="left")
df_test.head(10)

,user_id,merchant_id,prob,age_range,gender,categories,one_clicks,shopping_carts,purchase_times,favourite_times
0,163968,4605,NaN,0.0,0.0,1,1,0,1,0
1,360576,1581,NaN,2.0,2.0,4,5,0,5,0
2,98688,1964,NaN,6.0,0.0,1,5,0,1,0
3,98688,3645,NaN,6.0,0.0,1,10,0,1,0
4,295296,3361,NaN,2.0,1.0,4,47,0,1,2
5,33408,98,NaN,2.0,0.0,1,9,0,1,1
6,230016,1742,NaN,5.0,1.0,1,11,0,2,0
7,164736,598,NaN,3.0,1.0,1,1,0,1,0
8,164736,1963,NaN,3.0,1.0,1,2,0,1,0
9,164736,2634,NaN,3.0,1.0,3,6,0,1,0


In [104]:
df_test = df_test.fillna(method='bfill')
df_test = df_test.fillna(method='ffill')

In [105]:
df_test.to_csv("预测集特征构建1.csv")

In [117]:
dataTest = spark.read.options(header='True', inferSchema='False', delimiter=',')\
.csv("预测集特征构建1.csv").rdd

In [118]:
headerTest = dataTest.first()

In [119]:
dataTest1 = dataTest.filter(lambda row:row != headerTest)


In [120]:
dataTest1.top(10)

[Row(_c0='99999', user_id='197797', merchant_id='2637', prob=None, age_range='6.0', gender='0.0', categories='1', one_clicks='2', shopping_carts='0', purchase_times='3', favourite_times='0'),
 Row(_c0='99998', user_id='131749', merchant_id='1538', prob=None, age_range='5.0', gender='0.0', categories='1', one_clicks='3', shopping_carts='0', purchase_times='1', favourite_times='0'),
 Row(_c0='99997', user_id='131749', merchant_id='908', prob=None, age_range='5.0', gender='0.0', categories='1', one_clicks='2', shopping_carts='0', purchase_times='1', favourite_times='0'),
 Row(_c0='99996', user_id='262309', merchant_id='747', prob=None, age_range='3.0', gender='0.0', categories='1', one_clicks='0', shopping_carts='0', purchase_times='1', favourite_times='0'),
 Row(_c0='99995', user_id='65701', merchant_id='3288', prob=None, age_range='7.0', gender='1.0', categories='1', one_clicks='1', shopping_carts='0', purchase_times='1', favourite_times='0'),
 Row(_c0='99994', user_id='327589', merchan

In [121]:
dataTest2 = dataTest1.collect()

In [150]:
testData = sc.parallelize(dataTest2).map(lambda x:LabeledPoint(label=1,features=x[4:]))

In [152]:
# print("预测集数量:{}".format(testData.count()))
# testData1 = testData1.map(lambda x:)
# testData.top(10)
# testData.count()
testData.first()

LabeledPoint(1.0, [2.0,2.0,4.0,5.0,0.0,5.0,0.0])

In [147]:
svmTotalCorrect = testData.map(lambda x: 1 if svm.predict(x.features)  else 0)

In [153]:
svmTotalCorrect.top(10)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [158]:
# schema = StructType([
#         # true代表不为空
#         StructField("prob", StringType(), True)
#     ])

# df = spark.createDataFrame(svmTotalCorrect, schema = schema)


In [165]:
# df1 = df.toPandas()
df = pd.DataFrame(svmTotalCorrect.collect())

In [166]:
df.to_csv("prob.csv")